## Import Dependencies

In [1]:
# Datetime manipulation
import datetime as dt

# Data manipulation
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 500, 'display.max_rows', 500, 'display.width', 1000)

# Plotting & data visualisation
import seaborn as sns
sns.set_style('darkgrid')
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

# Pre-config plotting style
%config InlineBackend.figure_format = 'retina'
matplotlib.rc('xtick', labelsize=12) 
matplotlib.rc('ytick', labelsize=12)

# Data pre-processing

In [2]:
weather = pd.read_csv('./datasets/weather.csv')

# `weather`

In [3]:
weather.head()

,Station,Date,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,Sunrise,Sunset,CodeSum,Depth,Water1,SnowFall,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed
0,1,2007-05-01,83,50,67,14,51,56,0,2,0448,1849,,0,M,0.0,0.00,29.10,29.82,1.7,27,9.2
1,2,2007-05-01,84,52,68,M,51,57,0,3,-,-,,M,M,M,0.00,29.18,29.82,2.7,25,9.6
2,1,2007-05-02,59,42,51,-3,42,47,14,0,0447,1850,BR,0,M,0.0,0.00,29.38,30.09,13.0,4,13.4
3,2,2007-05-02,60,43,52,M,42,47,13,0,-,-,BR HZ,M,M,M,0.00,29.44,30.08,13.3,2,13.4
4,1,2007-05-03,66,46,56,2,40,48,9,0,0446,1851,,0,M,0.0,0.00,29.39,30.12,11.7,7,11.9


In [4]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2944 entries, 0 to 2943
Data columns (total 22 columns):
Station        2944 non-null int64
Date           2944 non-null object
Tmax           2944 non-null int64
Tmin           2944 non-null int64
Tavg           2944 non-null object
Depart         2944 non-null object
DewPoint       2944 non-null int64
WetBulb        2944 non-null object
Heat           2944 non-null object
Cool           2944 non-null object
Sunrise        2944 non-null object
Sunset         2944 non-null object
CodeSum        2944 non-null object
Depth          2944 non-null object
Water1         2944 non-null object
SnowFall       2944 non-null object
PrecipTotal    2944 non-null object
StnPressure    2944 non-null object
SeaLevel       2944 non-null object
ResultSpeed    2944 non-null float64
ResultDir      2944 non-null int64
AvgSpeed       2944 non-null object
dtypes: float64(1), int64(5), object(16)
memory usage: 506.1+ KB


# Data Cleaning

## Weather Dataset

- `Date`
    - Convert to *datetime* object
- `Depart`
    - Duplicate from Station 1 to Station 2's missing records   
- `Sunrise`
    - Duplicate from Station 1 to Station 2's missing records
- `Sunset`
     - Duplicate from Station 1 to Station 2's missing records
- `CodeSum`
    - Change empty strings to 'Moderate' (as mentioned in data dictionary)
- ` Depth`
    - To drop as half of the data is missing
- `Water1` 
    - To drop as all the data is 'M'- Missing

### `Date` - convert to *datetime* object

In [5]:
# Convert Date column to a datetime object
weather['Date'] = pd.to_datetime(weather['Date'])

In [6]:
# Create year & month columns
weather['Year'] = weather['Date'].dt.year
weather['Month'] = weather['Date'].dt.month

### `Depart`, `Sunrise`,  `Sunset`, `Heat`, `Cool`,  `SnowFall`, `PrecipTotal` & `AvgSpeed`  - Duplicate from Station 1 to Station 2's missing records

In [7]:
# Replace missing values represented by 'M' or '-') with NaN
replace_dict = {'M': np.NaN, '-': np.NaN}                                                                                          
weather = weather.replace(replace_dict)

In [8]:
# List out features that are missing values every alternate row
# Meaning that the data is only recorded at Station 1, hence, we will fill Station 2 with Sation 1's values
station1_to_2 = ['Depart','Depth','Sunrise','Sunset','Heat','Cool','SnowFall','PrecipTotal','AvgSpeed']

# Fill NaN with prior entry from Station 1 for alternate rows
for col in station1_to_2:
    weather[col] = weather[col].fillna(method='ffill') 

In [9]:
weather.head()

,Station,Date,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,Sunrise,Sunset,CodeSum,Depth,Water1,SnowFall,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed,Year,Month
0,1,2007-05-01,83,50,67,14,51,56,0,2,0448,1849,,0,NaN,0.0,0.00,29.10,29.82,1.7,27,9.2,2007,5
1,2,2007-05-01,84,52,68,14,51,57,0,3,0448,1849,,0,NaN,0.0,0.00,29.18,29.82,2.7,25,9.6,2007,5
2,1,2007-05-02,59,42,51,-3,42,47,14,0,0447,1850,BR,0,NaN,0.0,0.00,29.38,30.09,13.0,4,13.4,2007,5
3,2,2007-05-02,60,43,52,-3,42,47,13,0,0447,1850,BR HZ,0,NaN,0.0,0.00,29.44,30.08,13.3,2,13.4,2007,5
4,1,2007-05-03,66,46,56,2,40,48,9,0,0446,1851,,0,NaN,0.0,0.00,29.39,30.12,11.7,7,11.9,2007,5


### `CodeSum` - fill empty strings to 'Moderate'

In [10]:
# Replace empty strings with 'Moderate'
weather['CodeSum'][weather['CodeSum'] == ' '] = 'Moderate'

/Users/jeromechua/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


### `WetBulb`, `StnPressure` , `SeaLevel`  - fill with the month's median

The missing values for `WetBulb`, `StnPressure` and `SeaLevel` are missing not because it was recorded only at Station 1 and not Station 2. Hence, we will be filling them with the year & month's median value respectively.

In [11]:
weather[weather.WetBulb.isnull()]

,Station,Date,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,Sunrise,Sunset,CodeSum,Depth,Water1,SnowFall,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed,Year,Month
848,1,2009-06-26,86,69,78,7,60,NaN,0,13,0418,1931,Moderate,0,NaN,0.0,0.00,NaN,29.85,6.4,4,8.2,2009,6
2410,1,2013-08-10,81,64,73,0,57,NaN,0,8,0454,1900,Moderate,0,NaN,0.0,0.00,NaN,30.08,5.3,5,6.5,2013,8
2412,1,2013-08-11,81,60,71,-2,61,NaN,0,6,0455,1859,RA,0,NaN,0.0,0.01,29.35,30.07,2.0,27,3.0,2013,8
2415,2,2013-08-12,85,69,77,2,63,NaN,0,12,0456,1858,RA,0,NaN,0.0,0.66,29.27,29.92,4.5,26,7.7,2013,8


In [12]:
# Find median values of June 2009 & August 2013 to replace WetBulb's null values
median_0609 = weather[(weather.Month == 6) & (weather.Year == 2009)]['WetBulb'].median()
median_0813 = weather[(weather.Month == 8) & (weather.Year == 2013)]['WetBulb'].median()

# Replace accordingly
weather.at[848, 'WetBulb'] = median_0609
weather.at[2410, 'WetBulb'] = median_0813
weather.at[2412, 'WetBulb'] = median_0813
weather.at[2415, 'WetBulb'] = median_0813

In [13]:
weather[weather.StnPressure.isnull()]

,Station,Date,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,Sunrise,Sunset,CodeSum,Depth,Water1,SnowFall,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed,Year,Month
87,2,2007-06-13,86,68,77,6,53,62,0,12,0416,1927,Moderate,0,NaN,0.0,0.00,NaN,NaN,7.0,5,8.6,2007,6
848,1,2009-06-26,86,69,78,7,60,59,0,13,0418,1931,Moderate,0,NaN,0.0,0.00,NaN,29.85,6.4,4,8.2,2009,6
2410,1,2013-08-10,81,64,73,0,57,65,0,8,0454,1900,Moderate,0,NaN,0.0,0.00,NaN,30.08,5.3,5,6.5,2013,8
2411,2,2013-08-10,81,68,75,0,55,63,0,10,0454,1900,Moderate,0,NaN,0.0,0.00,NaN,30.07,6.0,6,7.4,2013,8


In [14]:
# Find median values of June 2007/2009 & August 2013 to replace StnPressure's null values
median_0607 = weather[(weather.Month == 6) & (weather.Year == 2007)]['StnPressure'].median()
median_0609 = weather[(weather.Month == 6) & (weather.Year == 2009)]['StnPressure'].median()
median_0813 = weather[(weather.Month == 8) & (weather.Year == 2013)]['StnPressure'].median()

# Replace accordingly
weather.at[87, 'StnPressure'] = median_0607
weather.at[848, 'StnPressure'] = median_0609
weather.at[2410, 'StnPressure'] = median_0813
weather.at[2411, 'StnPressure'] = median_0813

In [15]:
weather[weather.SeaLevel.isnull()]

,Station,Date,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,Sunrise,Sunset,CodeSum,Depth,Water1,SnowFall,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed,Year,Month
87,2,2007-06-13,86,68,77,6,53,62,0,12,0416,1927,Moderate,0,NaN,0.0,0.00,29.33,NaN,7.0,5,8.6,2007,6
832,1,2009-06-18,80,61,71,1,63,67,0,6,0416,1929,RA BR,0,NaN,0.0,0.12,29.08,NaN,6.7,16,7.9,2009,6
994,1,2009-09-07,77,59,68,1,59,62,0,3,0523,1817,BR,0,NaN,0.0,0.00,29.39,NaN,5.8,3,4.0,2009,9
1732,1,2011-09-08,75,57,66,0,53,59,0,1,0524,1815,RA,0,NaN,0.0,T,29.34,NaN,13.0,2,13.4,2011,9
1745,2,2011-09-14,60,48,54,-11,45,51,11,0,0530,1805,RA BR HZ FU,0,NaN,0.0,T,29.47,NaN,6.0,32,7.3,2011,9
1756,1,2011-09-20,74,49,62,0,54,58,3,0,0537,1753,MIFG BCFG BR,0,NaN,0.0,0.00,29.26,NaN,7.3,18,7.3,2011,9
2067,2,2012-08-22,84,72,NaN,-1,51,61,0,5,0506,1843,Moderate,0,NaN,0.0,0.00,29.39,NaN,4.7,19,5.8,2012,8
2090,1,2012-09-03,88,71,80,12,70,73,0,15,0519,1824,BR,0,NaN,0.0,0.00,29.17,NaN,4.6,6,4.4,2012,9
2743,2,2014-07-23,76,64,70,-7,56,61,0,5,0436,1920,Moderate,0,NaN,0.0,0.00,29.47,NaN,16.4,2,16.7,2014,7


In [16]:
# Find median values of June 2007/2009 & August 2013 to replace SeaLevel's null values
median_0607 = weather[(weather.Month == 6) & (weather.Year == 2007)]['SeaLevel'].median()
median_0609 = weather[(weather.Month == 6) & (weather.Year == 2009)]['SeaLevel'].median()
median_0909 = weather[(weather.Month == 9) & (weather.Year == 2009)]['SeaLevel'].median()
median_0911 = weather[(weather.Month == 9) & (weather.Year == 2011)]['SeaLevel'].median()
median_0812 = weather[(weather.Month == 8) & (weather.Year == 2012)]['SeaLevel'].median()
median_0912 = weather[(weather.Month == 9) & (weather.Year == 2012)]['SeaLevel'].median()
median_0714 = weather[(weather.Month == 7) & (weather.Year == 2014)]['SeaLevel'].median()

# Replace accordingly
weather.at[87, 'SeaLevel'] = median_0607
weather.at[832, 'SeaLevel'] = median_0609
weather.at[994, 'SeaLevel'] = median_0909
weather.at[1732, 'SeaLevel'] = median_0911
weather.at[1745, 'SeaLevel'] = median_0911
weather.at[1756, 'SeaLevel'] = median_0911
weather.at[2067, 'SeaLevel'] = median_0812
weather.at[2090, 'SeaLevel'] = median_0912
weather.at[2743, 'SeaLevel'] = median_0714

### `PrecipTotal`'s ['T' values to be replaced with 0.005](https://www.chicagotribune.com/news/ct-xpm-2011-04-16-ct-wea-0417-asktom-20110416-story.html)

In [17]:
weather.PrecipTotal.replace(to_replace='  T', value=.005, inplace=True)

### Rectify weather data types

In [18]:
# Convert following data types to float
weather = weather.astype({'Tavg': np.float,
                          'Depth': np.float,
                          'Depart': np.float, 
                          'WetBulb': np.float, 
                          'Heat': np.float, 
                          'Cool': np.float,
                          'PrecipTotal': np.float, 
                          'StnPressure': np.float,
                          'SeaLevel': np.float,
                          'AvgSpeed': np.float
                         }, inplace=True)

weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2944 entries, 0 to 2943
Data columns (total 24 columns):
Station        2944 non-null int64
Date           2944 non-null datetime64[ns]
Tmax           2944 non-null int64
Tmin           2944 non-null int64
Tavg           2933 non-null float64
Depart         2944 non-null float64
DewPoint       2944 non-null int64
WetBulb        2944 non-null float64
Heat           2944 non-null float64
Cool           2944 non-null float64
Sunrise        2944 non-null object
Sunset         2944 non-null object
CodeSum        2944 non-null object
Depth          2944 non-null float64
Water1         0 non-null float64
SnowFall       2944 non-null object
PrecipTotal    2944 non-null float64
StnPressure    2944 non-null float64
SeaLevel       2944 non-null float64
ResultSpeed    2944 non-null float64
ResultDir      2944 non-null int64
AvgSpeed       2944 non-null float64
Year           2944 non-null int64
Month          2944 non-null int64
dtypes: datetime64[

### To consider dropping the following features

- `SnowFall`
- `Depth`
- `Water1`
- `Sunrise`
- `Sunset`

In [19]:
# # Export file to csv
# weather.to_csv('weather_clean.csv', index=False)